# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum_sqrt_value = 0

for sqrt_value in generator:
    print(sqrt_value)
    sum_sqrt_value += sqrt_value

print(sum_sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data = list(people_1())

sum_of_ages = sum(person["Age"] for person in data)
print(len(data))
print(sum_of_ages)

# for person in people_1():
#     print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data.extend(people_2())
print(len(data))



sum_of_ages = sum(person["Age"] for person in data)

print(sum_of_ages)

# for person in people_2():
#     print(person)


5
140
11
353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
#Install the dependencies
# %%capture
%pip install dlt[duckdb]
%pip install streamlit

In [5]:
# to do: homework :)
import dlt

pipeline = dlt.pipeline(pipeline_name="people_data", destination='duckdb', dataset_name='personal_data')

info = pipeline.run(people_1, table_name='persons', primary_key='id', write_disposition='replace')


# print(info)



In [6]:
pipeline = dlt.pipeline(pipeline_name="people_data", destination='duckdb', dataset_name='personal_data')
pipeline.run(people_2, table_name="persons",
					write_disposition="merge",
					primary_key="id")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x00000220781C2CD0>, metrics={'1707415178.866187': [{'started_at': DateTime(2024, 2, 8, 17, 59, 39, 328429, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 8, 17, 59, 39, 868032, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///c:\\Users\\amrna\\OneDrive\\Desktop\\people_data.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='personal_data', loads_ids=['1707415178.866187'], load_packages=[LoadPackageInfo(load_id='1707415178.866187', package_path='C:\\Users\\amrna\\.dlt\\pipelines\\people_data\\load\\loaded\\1707415178.866187', state='loaded', schema=Schema people_data at 2338507450704, schema_update={}, completed_at=DateTime(2024, 2, 8, 17, 59, 39, 843620, tzinfo=Timezone('UTC')), jobs={'new_jobs': [], 'completed_jobs': [LoadJobI

In [9]:
import duckdb

# setup connection to pipeline
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# duckdb to display tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n persons table below:")

persons = conn.sql("SELECT * FROM persons").df()
display(persons)

print("\n\n\n stream_download table below:")

total_age = conn.sql("SELECT SUM(age) FROM persons").df()
display(total_age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons             │
└─────────────────────┘




 persons table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707415175.3786669,ylQ1Gb4q65hjnQ,None
1,2,Person_2,27,City_A,1707415175.3786669,3g7xP0BnOg5U3A,None
2,5,Person_5,35,City_B,1707415178.866187,0qndJpnw4LJzjw,Job_5
3,7,Person_7,37,City_B,1707415178.866187,TfOZJDYBWfubMQ,Job_7
4,4,Person_4,34,City_B,1707415178.866187,DWMccHgnKGZJlQ,Job_4
5,8,Person_8,38,City_B,1707415178.866187,SW+Jk9+ONDsjmQ,Job_8
6,3,Person_3,33,City_B,1707415178.866187,tGbx17v+RbzwEg,Job_3
7,6,Person_6,36,City_B,1707415178.866187,LqZz1Co/BBuhpQ,Job_6





 stream_download table below:


,sum(age)
0,266.0
